# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataset to get an overview
print(data.head())

# Check the basic statistics of the 'id' column to see how many viewers also clicked
viewers = data[data['action'] == 'view']['id'].nunique()
clickers = data[data['action'] == 'click']['id'].nunique()
overlap = data[(data['action'] == 'view') & (data['id'].isin(data[data['action'] == 'click']['id']))]['id'].nunique()

print(f"Number of viewers: {viewers}")
print(f"Number of clickers: {clickers}")
print(f"Number of viewers who also clicked: {overlap}")

# Check for anomalies: Did anyone click who didn't view?
click_without_view = data[(data['action'] == 'click') & (~data['id'].isin(data[data['action'] == 'view']['id']))]
print(f"Number of clicks without views: {click_without_view.shape[0]}")

# Check for overlap between control and experiment groups
control_group = data[data['group'] == 'control']['id'].unique()
experiment_group = data[data['group'] == 'experiment']['id'].unique()
overlap_group = set(control_group).intersection(experiment_group)

print(f"Number of users in both control and experiment groups: {len(overlap_group)}")


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Number of viewers: 6328
Number of clickers: 1860
Number of viewers who also clicked: 1860
Number of clicks without views: 0
Number of users in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
import pandas as pd
import scipy.stats as stats

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Calculate the click-through rates for experimental and control groups
experimental_clicks = data[(data['group'] == 'experiment') & (data['action'] == 'click')]['id'].nunique()
control_clicks = data[(data['group'] == 'control') & (data['action'] == 'click')]['id'].nunique()

experimental_total = data[data['group'] == 'experiment']['id'].nunique()
control_total = data[data['group'] == 'control']['id'].nunique()

p_exp = experimental_clicks / experimental_total
p_ctrl = control_clicks / control_total

# Perform a z-test for proportions
p_pool = (experimental_clicks + control_clicks) / (experimental_total + control_total)
z_score = (p_exp - p_ctrl) / ((p_pool * (1 - p_pool) * (1 / experimental_total + 1 / control_total)) ** 0.5)
p_value = 1 - stats.norm.cdf(z_score)

# Set the significance level (alpha)
alpha = 0.05

# Compare p-value to alpha
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is more effective than the control group.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the groups.")


Reject the null hypothesis. The experimental homepage is more effective than the control group.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [3]:
import pandas as pd
import scipy.stats as stats

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Calculate the click-through rates for experimental and control groups
experimental_clicks = data[(data['group'] == 'experiment') & (data['action'] == 'click')]['id'].nunique()
control_clicks = data[(data['group'] == 'control') & (data['action'] == 'click')]['id'].nunique()

experimental_total = data[data['group'] == 'experiment']['id'].nunique()
control_total = data[data['group'] == 'control']['id'].nunique()

p_exp = experimental_clicks / experimental_total
p_ctrl = control_clicks / control_total

# Calculate the expected number of clicks for the experiment group
expected_clicks_exp = p_ctrl * experimental_total

# Calculate the variance for the number of successes in a binomial variable
variance = experimental_total * p_ctrl * (1 - p_ctrl)

# Calculate the standard deviation
std_dev = variance ** 0.5

# Calculate the z-score
z_score = (experimental_clicks - expected_clicks_exp) / std_dev

# Calculate a two-tailed p-value using the normal distribution
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

# Set the significance level (alpha)
alpha = 0.05

# Compare p-value to alpha
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is more effective than the control group.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the groups.")


Reject the null hypothesis. The experimental homepage is more effective than the control group.


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.